In [ ]:
%pip install gymnasium stable-baselines3 torch
%pip install stable-baselines3[extra]
%pip install sb3-contrib
%pip install torch torchvision
%pip install numpy protobuf onnx onnxruntime
%pip install onnx


# 強化学習モデルの学習 (main.py)

このセルでは、DQNアルゴリズムを用いて、`CartPole-v1`環境でモデルを学習させます。

In [ ]:
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
import torch
from cat_toy_env import CatToyEnv

In [ ]:
env_kwargs=dict(render_mode=None, max_steps=1000)

# 1個だけ環境を作る（並列ではなく）
env_preview = CatToyEnv(**env_kwargs)

obs = env_preview.reset()

# 観測のshapeを確認
print("観測の形:", obs)
print("観測の中身:", obs)
# 学習用環境（4並列）
env_learning = make_vec_env(CatToyEnv, n_envs=4, env_kwargs=env_kwargs)

# 評価用環境（1つ）
env_eval = make_vec_env(CatToyEnv, n_envs=1, env_kwargs=env_kwargs)

In [ ]:
model_learning = DQN(
    "MlpPolicy",
    env_learning,
    verbose=1,
    learning_starts=1000,
    buffer_size=10000,
    exploration_fraction=0.2,
    exploration_final_eps=0.01,
    exploration_initial_eps=1.0,
    target_update_interval=1000,
    train_freq=8,
    gradient_steps=2,
    batch_size=64,
    gamma=0.99,
    learning_rate=1e-4,
    policy_kwargs=dict(net_arch=[256, 256])
)


In [ ]:
# モデルの学習
model_learning.learn(total_timesteps=200000, progress_bar=False, log_interval=10)

# モデルの評価（評価環境は1つ）
mean_reward, std_reward = evaluate_policy(model_learning, env_eval, n_eval_episodes=10)
print(f"mean_reward: {mean_reward:.2f} +/- {std_reward:.2f}")


In [ ]:
# モデルの保存
model_learning.save("cat_dqn")


In [ ]:
# モデルのロード
loaded_model = DQN.load("cat_dqn")

# ロードしたモデルの評価
mean_reward, std_reward = evaluate_policy(loaded_model, env_eval, n_eval_episodes=10)
print(f"loaded_mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
# 入力の2つのTensorを結合
toy = torch.randn(1, 2)
cat = torch.randn(1, 2)
concat_input = torch.cat([toy, cat], dim=1)  # shape: (1, 4)

# エクスポート対象モデル（例: policyネットワーク）
policy_net = loaded_model.policy.q_net

# ONNXエクスポート
torch.onnx.export(
    policy_net,
    concat_input,  # ← dictではなく単一Tensor
    "cat_dqn_policy.onnx",
    export_params=True,
    opset_version=11,
    input_names=["obs"],
    output_names=["q_values"],
    dynamic_axes={
        "obs": {0: "batch_size"},
        "q_values": {0: "batch_size"}
    }
)


In [ ]:
# 環境のクローズ
env_learning.close()
env_eval.close()

# 学習済みモデルの使用 (play.py)

このセルでは、学習済みのモデルをロードし、`CartPole-v1`環境でエージェントがどのように行動するかを観察します。

In [ ]:
import gymnasium as gym
from stable_baselines3 import DQN
import time
from cat_toy_env import CatToyEnv

In [ ]:
env_kwargs=dict(render_mode="", max_steps=1000, cat_speed = 2)

# 環境の作成
env = CatToyEnv(**env_kwargs)

# モデルのロード
model_playing = DQN.load("cat_dqn")

In [ ]:
# エピソードの実行
obs, info = env.reset()
done = False
while not done:
    action, _states = model_playing.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    print("観測:", obs)
    done = terminated or truncated
    env.render()  # 環境の描画
    #time.sleep(0.001) # 0.01秒待機

In [ ]:
# 環境のクローズ
env.close()